In [ ]:
print('Wheel Braking System .... SFHA')

Wheel Braking System .... SFHA


# Importing necessary modules


In [ ]:
import spacy
from sklearn import svm
nlp = spacy.load("en_core_web_sm")
import numpy as np
import pandas as pd


# Reading CSV_file

In [ ]:
df = pd.read_csv('WBS.csv')
df.head()

,Function,Failure Conditon,Failure Condition B,Phase,Effect on Crew,Effect on Pilot,Effect on Occupants,DAL
0,Decelerate Aircraft using wheel braking,Total Loss of Wheel Braking,Annuciated,Landing,NaN,NaN,NaN,Hazardous
1,Decelerate Aircraft using wheel braking,Total Loss of Wheel Braking,Unannuciated,Landing,NaN,NaN,NaN,Hazardous
2,Decelerate Aircraft using wheel braking,Total Loss of Wheel Braking,Unannuciated,Rejected Takeoff,NaN,NaN,NaN,Hazardous
3,Decelerate Aircraft using wheel braking,Partial Symmetric Loss of Loss of Wheel Braking,Unannuciated,Landing,NaN,NaN,NaN,Hazardous
4,Decelerate Aircraft using wheel braking,Partial Symmetric Loss of Loss of Wheel Braking,annuciated,Landing,NaN,NaN,NaN,Major


# Data Cleaning


In [ ]:
df.drop(columns=['Effect on Crew','Effect on Pilot', 'Effect on Occupants' ], inplace= True)
df.head()

,Function,Failure Conditon,Failure Condition B,Phase,DAL
0,Decelerate Aircraft using wheel braking,Total Loss of Wheel Braking,Annuciated,Landing,Hazardous
1,Decelerate Aircraft using wheel braking,Total Loss of Wheel Braking,Unannuciated,Landing,Hazardous
2,Decelerate Aircraft using wheel braking,Total Loss of Wheel Braking,Unannuciated,Rejected Takeoff,Hazardous
3,Decelerate Aircraft using wheel braking,Partial Symmetric Loss of Loss of Wheel Braking,Unannuciated,Landing,Hazardous
4,Decelerate Aircraft using wheel braking,Partial Symmetric Loss of Loss of Wheel Braking,annuciated,Landing,Major


In [ ]:
df.shape

(14, 5)

# Simplifying text

In [ ]:

# Replace spaces in column names with underscores
df.columns = df.columns.str.replace(' ', '_')

# Lowercase all column names and values in string columns
df = df.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

# Remove special characters from column names
df.columns = df.columns.str.replace('[^\w\s]', '')

# # Remove special characters and spaces from values in string columns
# df = df.applymap(lambda x: ''.join(e for e in str(x) if e.isalnum() or e.isspace()) if isinstance(x, str) else x)

# Display the preprocessed Data
df.head()


<ipython-input-5-fea379221998>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[^\w\s]', '')


,Function_,Failure_Conditon,Failure_Condition_B,Phase,DAL
0,decelerate aircraft using wheel braking,total loss of wheel braking,annuciated,landing,hazardous
1,decelerate aircraft using wheel braking,total loss of wheel braking,unannuciated,landing,hazardous
2,decelerate aircraft using wheel braking,total loss of wheel braking,unannuciated,rejected takeoff,hazardous
3,decelerate aircraft using wheel braking,partial symmetric loss of loss of wheel braking,unannuciated,landing,hazardous
4,decelerate aircraft using wheel braking,partial symmetric loss of loss of wheel braking,annuciated,landing,major


# Dividing Data into X and y

In [ ]:

X = df.drop('DAL', axis = 'columns')
X.head()
y = df.DAL
y.head()

0    hazardous
1    hazardous
2    hazardous
3    hazardous
4        major
Name: DAL, dtype: object

# Splitting into Train and Test sections

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state = 10)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(11, 4) (11,) (3, 4) (3,)


# Converting text to docs and then to vectors

In [ ]:
for column in X_train.columns:
  docs = [nlp(text) for text in (X_train[column])]
  # print(docs)
  x1_vecs = []
  trf = spacy.load('en_core_web_sm')
  for doc in trf.pipe(X_train[column]):
    a = doc.vector
    x1_vecs.append(a)
  print(docs)
  # print(x1_vecs)

[decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking]
[asymmetrical loss of wheel braking, asymmetrical loss of wheel braking, total loss of wheel braking, inadvertent wheel brake application, partial symmetric loss of loss of wheel braking, inadvertent wheel brake application, inadvertent wheel brake application, total loss of wheel braking, total loss of wheel braking, partial symmetric loss of loss of wheel braking, inadvertent wheel brake application]
[wheel braking system loss only, whell braking system loss only, unannuciated, with all wheels locked, unannuciated, all wheels locked or not 

# Using SVC linear Classifier

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(x1_vecs,y_train)

for column in X_test.columns:
  docs = [nlp(text) for text in (X_test[column])]
  # print(docs)
  x2_vecs = []
  trf = spacy.load('en_core_web_sm')
  for doc in trf.pipe(X_test[column]):
    a = doc.vector
    x2_vecs.append(a)
  print(docs)

[decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking]
[partial symmetric loss of loss of wheel braking, asymmetrical loss of wheel braking, inadvertent wheel brake application]
[unannuciated, wheel braking system and loss of rudder or nose wheel steering, undetected on one wheel without wheel locking]
[landing, landing, takeoff]


In [ ]:
results = clf.predict(x2_vecs)
print(results)
print(y_test)

['hazardous' 'hazardous' 'minor']
3        hazardous
7        hazardous
12    catastrophic
Name: DAL, dtype: object


# Using Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:

clf = DecisionTreeClassifier()
clf.fit(x1_vecs,y_train)

for column in X_test.columns:
  docs = [nlp(text) for text in (X_test[column])]
  # print(docs)
  x2_vecs = []
  trf = spacy.load('en_core_web_sm')
  for doc in trf.pipe(X_test[column]):
    a = doc.vector
    x2_vecs.append(a)
  print(docs)

[decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking]
[partial symmetric loss of loss of wheel braking, asymmetrical loss of wheel braking, inadvertent wheel brake application]
[unannuciated, wheel braking system and loss of rudder or nose wheel steering, undetected on one wheel without wheel locking]
[landing, landing, takeoff]


In [ ]:
results = clf.predict(x2_vecs)
print(results)
print(y_test)

['hazardous' 'hazardous' 'minor']
3        hazardous
7        hazardous
12    catastrophic
Name: DAL, dtype: object


# Using Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x1_vecs,y_train)

for column in X_test.columns:
  docs = [nlp(text) for text in (X_test[column])]
  # print(docs)
  x2_vecs = []
  trf = spacy.load('en_core_web_sm')
  for doc in trf.pipe(X_test[column]):
    a = doc.vector
    x2_vecs.append(a)
  print(docs)

[decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking]
[partial symmetric loss of loss of wheel braking, asymmetrical loss of wheel braking, inadvertent wheel brake application]
[unannuciated, wheel braking system and loss of rudder or nose wheel steering, undetected on one wheel without wheel locking]
[landing, landing, takeoff]


In [ ]:
results = clf.predict(x2_vecs)
print(results)
print(y_test)

['major' 'major' 'minor']
3        hazardous
7        hazardous
12    catastrophic
Name: DAL, dtype: object


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x1_vecs,y_train)

for column in X_test.columns:
  docs = [nlp(text) for text in (X_test[column])]
  # print(docs)
  x2_vecs = []
  trf = spacy.load('en_core_web_sm')
  for doc in trf.pipe(X_test[column]):
    a = doc.vector
    x2_vecs.append(a)
  print(docs)

[decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking]
[partial symmetric loss of loss of wheel braking, asymmetrical loss of wheel braking, inadvertent wheel brake application]
[unannuciated, wheel braking system and loss of rudder or nose wheel steering, undetected on one wheel without wheel locking]
[landing, landing, takeoff]


In [ ]:
results = clf.predict(x2_vecs)
print(results)
print(y_test)

['hazardous' 'hazardous' 'minor']
3        hazardous
7        hazardous
12    catastrophic
Name: DAL, dtype: object


# Shuffle Split Approach

In [ ]:
for column in X.columns:
  docs = [nlp(text) for text in (X[column])]
  # print(docs)
  x_vecs = []
  trf = spacy.load('en_core_web_sm')
  for doc in trf.pipe(X[column]):
    a = doc.vector
    x_vecs.append(a)
  print(docs)

[decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking, decelerate aircraft using wheel braking]
[total loss of wheel braking, total loss of wheel braking, total loss of wheel braking, partial symmetric loss of loss of wheel braking, partial symmetric loss of loss of wheel braking, partial symmetric loss of loss of wheel braking, asymmetrical loss of wheel braking, asymmetrical loss of wheel braking, asymmetrical loss of wheel braking, inadvertent wheel brake application, inadvertent wheel brake application, inadvert

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import ShuffleSplit, cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
cross_val_score(SVC(), x_vecs, y, cv=cv)

array([0.        , 0.33333333, 0.66666667, 0.        , 0.33333333])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit, cross_val_score


cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
cross_val_score(LogisticRegression(), x_vecs, y, cv=cv)

array([0.        , 0.33333333, 0.66666667, 0.        , 0.33333333])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ShuffleSplit, cross_val_score


cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
cross_val_score(DecisionTreeClassifier(), x_vecs, y, cv=cv)

array([0.        , 0.33333333, 0.66666667, 0.        , 0.33333333])